In [ ]:
import re
import numpy as np
import pandas as pd
import json

In [ ]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 50)

In [ ]:
log_filename = '../../../tmp/test-longevity-mp43.log'

In [ ]:
log_filename = '../../../tmp/test-longevity-mpegts3.log'

In [ ]:
records = []
with open(log_filename) as file:
    for line in file:
        m = re.match('{.*', line)
        if m:
            rec1 = json.loads(line)
            rec = rec1['fields'] 
            rec['level'] = rec1['level']
            rec['target'] = rec1['target']
            records += [rec]

In [ ]:
df = pd.DataFrame(records)

In [ ]:
if "prev_pts" not in df:
    df["prev_pts"] = np.nan
if "time_delta" not in df:
    df["time_delta"] = np.nan

In [ ]:
df["pts"] = pd.to_datetime(df["pts"], errors="coerce")
df["prev_pts"] = pd.to_datetime(df["prev_pts"], errors="coerce")

In [ ]:
df["ms_from_prev"] = df.pts.diff() / pd.Timedelta(1, "millisecond")

In [ ]:
df.info()

In [ ]:
df.head(4)

In [ ]:
df[df.level.isin(["WARN","DEBUG"])].groupby(["description", "probe_name"]).size()

In [ ]:
df[df.description.isin(["PTS is missing"])][["pts", "time_delta", "probe_name"]]

In [ ]:
df[df.description.isin(["PTS is decreasing"])][["pts", "time_delta", "probe_name"]]

In [ ]:
df.iloc[300:400][df.probe_name.isin(["1-pravegasrc"])][["pts", "ms_from_prev", "prev_pts", "time_delta", "probe_name", "flags", "size", "description"]]

In [ ]:
df2 = df[df.description.isin(["buffer"]) & df.probe_name.isin(["1-pravegasrc"])].copy()

In [ ]:
df2["ms_from_prev"] = df2.pts.diff() / pd.Timedelta(1, "millisecond")

In [ ]:
df2[["pts", "ms_from_prev", "element", "pad", "flags", "size", "description"]].head(200)

In [ ]:
df2.ms_from_prev.value_counts()

In [ ]:
df2 = df[df.description.isin(["buffer"]) & ((df.element=="pravegasrc") | ((df.element=="h264parse") & (df["pad"]=="sink")))].copy()

In [ ]:
df2["ms_from_prev"] = df2.pts.diff() / pd.Timedelta(1, "millisecond")

In [ ]:
df2[["pts", "ms_from_prev", "element", "pad", "flags", "size", "description"]].head(200)

In [ ]:
df2[df2.element=="h264parse"].ms_from_prev.value_counts()

In [ ]:
df2.ms_from_prev.value_counts()